# Data Collection
Load original training data. <br>
Need to unzip train.zip and its articles into a directory called `train_articles` <br>

In [1]:
import pandas as pd
import os
from tqdm import tqdm_notebook as tqdm
import random


In [5]:
SOURCE = "utils/"
DEST = "data/"
HOME = "./"

In [6]:
def get_original(data_path):
    print("Getting original...")
    data_original = pd.read_json(DEST+"/train.json")
    for idx, row in tqdm(data_original.iterrows(), total=len(data_original)):
        articles_content = []
        for a in row.related_articles:
            articles_content.append(open(data_path + "train_articles/" + str(a) + ".txt", "r").read())
        data_original.at[idx, "related_articles"] =  articles_content
    return data_original
    
def get_scrapped(data_path, ids_original):
    print("Getting scrapped...")
    max_id = max(ids_original)+1
    print("max_id:",max_id)
    data_scrapped = pd.read_json(data_path+"/agg_dataset.json").drop(columns=['site','accessed_date'])
    data_scrapped = data_scrapped.dropna(subset=['_id', 'label'])
    data_scrapped["id"] = range(max_id, max_id+len(data_scrapped))
    data_scrapped["claim"] = data_scrapped["_id"]
    data_scrapped = data_scrapped.drop(columns=["_id"])
    data_scrapped["label"] = data_scrapped["label"].astype("int")
    return data_scrapped

def get_all_data(data_path):
    original_data = get_original(data_path)
    scrapped_data = get_scrapped(data_path, original_data.id)
    print("Concatenating datasets...")

    all_data = pd.concat([original_data, scrapped_data], sort=False)
    
    # now shuffle the related articles list
    for idx, row in tqdm(all_data.iterrows(), total = len(all_data)):
        random.shuffle(row.related_articles)

    return all_data.sample(frac=1).reset_index() # shuffle complete output    

In [7]:
all_data = get_all_data(DEST)

Getting original...


FileNotFoundError: [Errno 2] No such file or directory: 'data/train_articles/122094.txt'

In [56]:
all_data.to_json(DEST+"all_training_data.json")